### GPT for Learning how to play 2048

Inspired by [GPT from scratch](https://www.youtube.com/watch?v=kCc8FmEb1nY)

#### Explore the data

In [1]:
with open('games.txt', 'r') as f:
    text = f.read()

print(text[:1000])
print(len(text))

00000201000000000000000000000000,00000000000000000000000000010201,00010201000000000000000000000001,00010202010000000000000000000000,01030000010000010000000000000000,02030001000000000000000000010000,02030001000100000000000001000000,02030001010100000000000000010000,02030001010200000000000000020000,02030001010300000000000001000000,02040001020000000000010000000000,03040101000000000000000000000100,03040201000000000000000000010000,03040201010100000000000000000000,03040201020000000002000000000000,03040201020200010000000000000000,03040202020200010000000000000000,03040300030100000001000000000000,04040300000200000000000000020000,00000000000000000004010004030300,00000000000000000000040101000404,00000000000000010000040100000105,00000000000000000001040200000105,00000000000000000001040200010105,00000000010000000001040200000205,00000100000000000000040201010205,00010001000000000000040200020205,00000002010000000000040200000305,00000100000000000000040301000305,00000000000001000100040301000305,0000010000

#### Tokenize

Each tile is represented by 2 digit, 0-led int. Will convert tile into regular int. 

Each state is separated by comma, each game by ';\n'. Will convert these into int as well

In [2]:
mapping = {',':17, ';\n':18}
inv_mapping = {v: k for k, v in mapping.items()}

def encode(s:str)-> list[str]:
    out = []
    num_splits = len(s.split(f';\n'))
    for i, game in enumerate(s.split(f';\n')): 
        for state in game.split(',')[:-1]:
            enc_state = [int(''.join(state[i:i+2])) for i in range(0, len(state), 2)]
            enc_state.append(mapping[','])
            out += enc_state
        out.append(mapping[';\n']) if i < num_splits-1 else None
    
    return out

def decode(l:list[int]) -> str:
    s = ''
    for char in l:
        if char < 10:
            s += '0'+ str(char)
        elif char < min(mapping.values()): # hard coded, should fix
            s += str(char)
        else:
            s += inv_mapping[char]

    return s        


print(encode('02000000020303010607060507081011,01000000030303010607060507081011,;\n01000000030303010607060507081011,'))
print(decode(encode('02000000020303010607060507081011,01000000030303010607060507081011,;\n01000000030303010607060507081011,')))


[2, 0, 0, 0, 2, 3, 3, 1, 6, 7, 6, 5, 7, 8, 10, 11, 17, 1, 0, 0, 0, 3, 3, 3, 1, 6, 7, 6, 5, 7, 8, 10, 11, 17, 18, 1, 0, 0, 0, 3, 3, 3, 1, 6, 7, 6, 5, 7, 8, 10, 11, 17]
02000000020303010607060507081011,01000000030303010607060507081011,;
01000000030303010607060507081011,


#### Load data

Get train test split

Set up batches

In [36]:
import torch

torch.manual_seed(1748)

data = torch.tensor(encode(text), dtype=torch.long)

batch_size = 64
block_size = 34 * 3 # two boards * n
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_iters = 5000
eval_iters = 200
eval_interval = num_iters//10
n_embed = 384
n_head = 6
n_layer = 6
dropout= 0.2
learning_rate = 3e-4
print(device)

# get first game in last 20% of data
n = int(0.8*len(data))
while(data[n] != mapping[';\n']):
    n += 1
n += 2

train_data = data[:n]
test_data = data[n:]
print(n, len(data), len(train_data), len(test_data))

def get_batch(split:bool=0)-> list[torch.Tensor]:
    # split == 0: train, 1: test
    data = train_data if split == 0 else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y

x, y = get_batch()

print(x)

print(y)



cuda
1816089 2223064 1816089 406975
tensor([[ 0,  6, 17,  ...,  2,  1,  1],
        [ 6,  1,  0,  ...,  1,  5,  3],
        [ 2,  5,  5,  ...,  8,  4,  1],
        ...,
        [ 2,  6,  4,  ...,  1,  1,  2],
        [ 5,  1,  0,  ...,  5,  2,  3],
        [ 2,  2,  5,  ...,  0,  5,  1]], device='cuda:0')
tensor([[ 6, 17,  0,  ...,  1,  1,  0],
        [ 1,  0,  1,  ...,  5,  3,  3],
        [ 5,  5,  2,  ...,  4,  1,  1],
        ...,
        [ 6,  4,  3,  ...,  1,  2,  2],
        [ 1,  0,  0,  ...,  2,  3,  2],
        [ 2,  5,  6,  ...,  5,  1,  0]], device='cuda:0')


#### Bigram Model

Loss With multi-self attention: 4900: Train loss: 1.2557, Val loss: 1.1633

Residual Blocks: 4900: Train loss: 1.2156, Val loss: 1.1171
Seems to have learned state size

Scaling up model 4500: Train loss: 0.4378, Val loss: 0.4096

In [37]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1748)


class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4* n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed//n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramModel(nn.Module):

    def __init__(self, vocab_size=19):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T)
        for  _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:] # last time step, (B,C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx

    
model = BigramModel()
m = model.to(device)
logits, loss = m(x,y)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

torch.Size([6528, 19])
tensor(3.0377, device='cuda:0', grad_fn=<NllLossBackward0>)
00041615020706000607110606030403070412131214060400090004,160811001100031308051306011111;
03031512090100160602,0612131502100307;
0211031506020808070200;
020811001203,0811070016050903101209061313080515


In [38]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in [0, 1]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)



for iter in range(num_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'{iter}: Train loss: {losses[0]:.4f}, Val loss: {losses[1]:.4f}')
    xb, yb = get_batch(0)

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

res = decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist())
print(res)



0: Train loss: 3.0343, Val loss: 3.0404
500: Train loss: 0.8938, Val loss: 0.8033
1000: Train loss: 0.7611, Val loss: 0.6797
1500: Train loss: 0.6412, Val loss: 0.5678
2000: Train loss: 0.5742, Val loss: 0.5127
2500: Train loss: 0.5253, Val loss: 0.4721
3000: Train loss: 0.4925, Val loss: 0.4486
3500: Train loss: 0.4712, Val loss: 0.4349
4000: Train loss: 0.4528, Val loss: 0.4228
4500: Train loss: 0.4378, Val loss: 0.4096
0000000001050000020100,02020103000000000001000001020000,00020103000000000000010100000102,00010303000200000000000100000202,00000000000000000001030400020202,00000000010000000103040003000200,01000000


In [39]:
res = decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist())
print(res)
print([len(state) for state in res.split(',')])

000003010306040103071011,00010201000003010306040103071011,01020100030100010306040103071011,00000001010101000306040204071011,00010001000000020306040204071011,00000001000100020306040204071011,010001
[24, 32, 32, 32, 32, 32, 6]


In [72]:
def count_digits(n):
    count = 1
    while n // 10 >= 1:
        n /= 10 
        count += 1
    return count

def print_states(s):
    for i, game in enumerate(s.split(f';\n')): 
        for state in game.split(',')[:-1]:
            enc_state = [2**int(''.join(state[i:i+2])) for i in range(0, len(state), 2)]
            for i, tile in enumerate(enc_state):
                print(' ' * max(0, 5 - count_digits(tile)) + (str(tile) if tile != 1 else '0'), end=('\n' if (i+1) % 4 == 0 else ''))

            print('\n\n')
res = decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=1000)[0].tolist())
print(res)
print([len(state) for state in res.split(',')])
print(count_digits(0))
print_states(res)

0000000202040204070910,00000001000100000202040204070910,00010000000100010202040204070910,00000000000201010202040204070910,00000001020201000304020004070910,00000000020201000304020104070910,01000000030100000304020104070910,00000000010101000404020104070910,00000002010101000404020104070910,01000100000101020404020104070910,00000002000102020105020104070910,00000000010100030105030104070910,00000001000002030105030104070910,01000000020300010105030104070910,01000002000203010105030104070910,01020100020301000105030104070910,00020001020302000105030104070910,01000201000003020105030104070910,00000100000002020205040104070910,00000001000001030205040104070910,00000001000001030205040104070910,01000100010300000205040104070910,00000100020301000205040104070910,01000100020301000205040104070910,02000000020301010205040104070910,00000000030301010305040104070910,01000000000301000305040204070910,00000000010301010305040204070910,00010000000103020305040204070910,00000001000103020305040204070910,00010000010103010305